In [14]:
ACADEMIC_PATH = r'../../data/raw/academic_records.csv'
ADMISSION_PATH = r'../../data/raw/admission.csv'
academic_records = pd.read_csv(ACADEMIC_PATH)
admission = pd.read_csv(ADMISSION_PATH)

In [15]:
import pandas as pd
import numpy as np

def clean_data_pipeline(admission, academic_records):
    print("--- BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU ---")
    
    # =========================================================================
    # BƯỚC 1: XỬ LÝ ĐỊNH DẠNG & KIỂU DỮ LIỆU (Formatting)
    # =========================================================================
    print("Step 1: Formatting & Merging...")
    
    # 1.1. Chuẩn hóa MA_SO_SV về dạng String
    admission['MA_SO_SV'] = admission['MA_SO_SV'].astype(str)
    academic_records['MA_SO_SV'] = academic_records['MA_SO_SV'].astype(str)
    
    # 1.2. Chuyển đổi HOC_KY sang số nguyên (giả định hàm hoc_ky_to_code đã được định nghĩa bên ngoài)
    # Nếu chưa có hàm này, bạn cần định nghĩa logic convert (ví dụ: '20231' -> int)
    # academic_records['HOC_KY'] = academic_records['HOC_KY'].apply(hoc_ky_to_code)
    
    # Merge dữ liệu (Inner Join để chỉ lấy sinh viên có thông tin ở cả 2 bảng)
    df = pd.merge(academic_records, admission, on='MA_SO_SV', how='inner')
    
    # 1.3. Định dạng lại các cột số (Ép kiểu float/int)
    numeric_floats = ['GPA', 'CPA', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN']
    for col in numeric_floats:
        df[col] = pd.to_numeric(df[col], errors='coerce') 
        
    numeric_ints = ['TC_DANGKY', 'TC_HOANTHANH']
    for col in numeric_ints:
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Sắp xếp dữ liệu
    df = df.sort_values(by=['MA_SO_SV', 'HOC_KY']).reset_index(drop=True)

    # =========================================================================
    # BƯỚC 2: KIỂM TRA LOGIC & LÀM SẠCH NHIỄU (Sanity Checks)
    # =========================================================================
    print("Step 2: Sanity Checks & Logic Cleaning...")
    
    initial_rows = len(df)

    # 3.1. Logic Tín chỉ: Hoàn thành không được lớn hơn Đăng ký
    df['TC_HOANTHANH'] = np.minimum(df['TC_HOANTHANH'], df['TC_DANGKY'])
    
    # 3.2. Logic Điểm số (0 <= GPA/CPA <= 4.0)
    df['GPA'] = df['GPA'].clip(lower=0.0, upper=4.0)
    df['CPA'] = df['CPA'].clip(lower=0.0, upper=4.0)
    
    # -----------------------------------------------------------
    # [MỚI] 3.3. Logic Tuyển sinh: DIEM_TRUNGTUYEN >= DIEM_CHUAN
    # -----------------------------------------------------------
    # Loại bỏ các dòng mà điểm trúng tuyển nhỏ hơn điểm chuẩn.
    # Lưu ý: Các giá trị NaN (do lỗi format ở B1) cũng sẽ bị loại bỏ trong phép so sánh này.
    rows_before_score_filter = len(df)
    df = df[df['DIEM_TRUNGTUYEN'] >= df['DIEM_CHUAN']]
    dropped_score_rows = rows_before_score_filter - len(df)
    print(f" -> Đã loại bỏ {dropped_score_rows} dòng do Điểm trúng tuyển < Điểm chuẩn.")

    # 3.4. Xóa dữ liệu rác (TC_DANGKY = 0)
    rows_before_credit_filter = len(df)
    df = df[df['TC_DANGKY'] > 0].copy()
    dropped_credit_rows = rows_before_credit_filter - len(df)
    print(f" -> Đã loại bỏ {dropped_credit_rows} dòng rác (TC_DANGKY=0).")
    
    # Tổng kết
    total_dropped = initial_rows - len(df)
    print(f"--- HOÀN TẤT: Tổng cộng đã loại bỏ {total_dropped} dòng nhiễu. Kích thước data cuối: {df.shape} ---")
    
    return df

In [16]:
df = clean_data_pipeline(admission, academic_records)

--- BẮT ĐẦU QUY TRÌNH LÀM SẠCH DỮ LIỆU ---
Step 1: Formatting & Merging...
Step 2: Sanity Checks & Logic Cleaning...
 -> Đã loại bỏ 0 dòng do Điểm trúng tuyển < Điểm chuẩn.
 -> Đã loại bỏ 0 dòng rác (TC_DANGKY=0).
--- HOÀN TẤT: Tổng cộng đã loại bỏ 0 dòng nhiễu. Kích thước data cuối: (105726, 11) ---


### Các nhóm biến nâng cao

#### 1. Biến xu hướng & ổn định (Trend & Volatility)
- **GPA_Trend_Slope**: Đo lường xu hướng học tập của sinh viên (đang tiến bộ hay thụt lùi) qua nhiều kỳ. Slope > 0 là tiến bộ, < 0 là giảm sút.
- **GPA_Std_Dev_Accumulated**: Đo độ ổn định kết quả học tập. Std Dev cao nghĩa là sinh viên học thất thường, Std Dev thấp là học đều.
- **GPA_Last_3_Avg**: Trung bình GPA 3 kỳ gần nhất, giúp làm mượt dữ liệu, giảm ảnh hưởng của các kỳ bất thường.

#### 2. Biến so sánh tương đối (Relative/Cohort Features)
- **GPA_ZScore**: Chuẩn hóa điểm số theo kỳ, so sánh sinh viên với mặt bằng chung cùng khóa. Z-Score dương là vượt trội, âm là kém hơn.
- **Percentile Rank**: Xếp hạng phần trăm, cho biết sinh viên nằm ở đâu trong phân phối điểm số của kỳ đó (Top 10%, Bottom 10%,...).

#### 3. Biến tương tác (Interaction Features)
- **Admission_Score × First_Year_GPA**: Kiểm tra sự tương quan giữa năng lực đầu vào và kết quả năm đầu, phát hiện sinh viên giữ vững hay sa sút phong độ.
- **Gap_Duration × Credits_Failed**: Đo áp lực tích lũy, sinh viên vừa nghỉ học vừa nợ môn nhiều có nguy cơ bỏ học cao.
- **Total_Credits_Failed / Total_Credits_Registered**: Tỷ lệ nợ môn tích lũy, phản ánh rủi ro học tập dài hạn thay vì chỉ nhìn từng kỳ.

_Những biến này giúp mô hình hiểu sâu hơn về quá trình, sự ổn định và bối cảnh học tập của từng sinh viên, từ đó dự báo chính xác hơn._

In [17]:
import pandas as pd
import numpy as np
from scipy.stats import linregress

def calculate_slope(series):
    """
    Hàm phụ trợ: Tính hệ số góc (slope) của đường hồi quy tuyến tính cho một chuỗi điểm số.
    Trả về: > 0 (Đang tiến bộ), < 0 (Đang sa sút), 0 (Đi ngang/Không đủ dữ liệu)
    """
    y = series.dropna()
    if len(y) < 2: # Cần ít nhất 2 điểm để vẽ đường thẳng
        return 0
    x = np.arange(len(y))
    slope, intercept, r_value, p_value, std_err = linregress(x, y)
    return slope

def generate_advanced_features(df):
    """
    Hàm chính tạo ra các biến phái sinh nâng cao (Advanced Features).
    Input: DataFrame đã qua xử lý cơ bản (có GPA, MA_SO_SV, HOC_KY, v.v.)
    """
    print("🚀 Bắt đầu Feature Engineering nâng cao...")
    
    # Tạo bản sao để không ảnh hưởng dữ liệu gốc
    df_new = df.copy()
    
    # Đảm bảo dữ liệu được sort đúng để dùng các hàm window/shift
    df_new = df_new.sort_values(['MA_SO_SV', 'HOC_KY'])
    
    # ---------------------------------------------------------
    # 1. NHÓM BIẾN XU HƯỚNG & ỔN ĐỊNH (Trend & Volatility)
    # ---------------------------------------------------------
    print("- Đang tính toán xu hướng học tập (Trend)...")
    
    # Tính điểm trung bình 3 kỳ gần nhất (Rolling Average) để làm mượt dữ liệu
    # Dùng min_periods=1 để vẫn tính dù mới học 1 kỳ
    df_new['GPA_Last_3_Avg'] = df_new.groupby('MA_SO_SV')['GPA'].transform(
        lambda x: x.rolling(window=3, min_periods=1).mean()
    )
    
    # Tính độ ổn định (Độ lệch chuẩn tích lũy)
    # Std càng cao -> Học càng thất thường (khó dự đoán)
    df_new['GPA_Volatility_Accumulated'] = df_new.groupby('MA_SO_SV')['GPA'].expanding().std().reset_index(level=0, drop=True)
    df_new['GPA_Volatility_Accumulated'] = df_new['GPA_Volatility_Accumulated'].fillna(0) # Kỳ đầu tiên chưa có độ lệch
    
    # Tính độ dốc học tập (GPA Slope) - Sinh viên đang đi lên hay đi xuống?
    # Lưu ý: Hàm apply này có thể chạy hơi lâu nếu dữ liệu lớn (>100k dòng)
    df_new['GPA_Trend_Slope'] = df_new.groupby('MA_SO_SV')['GPA'].transform(
        lambda x: x.rolling(window=4, min_periods=2).apply(calculate_slope, raw=False)
    )
    df_new['GPA_Trend_Slope'] = df_new['GPA_Trend_Slope'].fillna(0)

    # ---------------------------------------------------------
    # 2. NHÓM BIẾN SO SÁNH TƯƠNG ĐỐI (Cohort Comparison)
    # ---------------------------------------------------------
    print("- Đang tính toán so sánh với bạn đồng lứa (Z-Score)...")
    
    # Gom nhóm theo từng kỳ (HOC_KY) để tính Mean và Std của kỳ đó
    semester_stats = df_new.groupby('HOC_KY')['GPA'].agg(['mean', 'std']).reset_index()
    semester_stats.columns = ['HOC_KY', 'Sem_Mean_GPA', 'Sem_Std_GPA']
    
    # Merge lại vào bảng chính
    df_new = df_new.merge(semester_stats, on='HOC_KY', how='left')
    
    # Tính Z-Score: (Điểm SV - Điểm TB Kỳ) / Độ lệch chuẩn Kỳ
    # Z-Score > 0: Giỏi hơn mặt bằng chung
    # Z-Score < 0: Kém hơn mặt bằng chung
    df_new['GPA_ZScore'] = (df_new['GPA'] - df_new['Sem_Mean_GPA']) / (df_new['Sem_Std_GPA'] + 1e-5) # Cộng epsilon để tránh chia cho 0
    
    # Xếp hạng phần trăm trong kỳ (Percentile)
    df_new['GPA_Percentile'] = df_new.groupby('HOC_KY')['GPA'].rank(pct=True)

    # ---------------------------------------------------------
    # 3. NHÓM BIẾN TƯƠNG TÁC (Interaction Features)
    # ---------------------------------------------------------
    print("- Đang tạo các biến tương tác (Interaction)...")
    
    # Tương tác: Áp lực nợ môn (Số tín chỉ chưa qua * Số kỳ đã nghỉ)
    # Giả định df đã có cột 'Gap_Duration' và 'TC_DANGKY', 'TC_HOANTHANH'
    if 'Gap_Duration' in df_new.columns and 'TC_DANGKY' in df_new.columns:
        # Tính số tín chỉ rớt trong kỳ
        df_new['TC_ROT'] = df_new['TC_DANGKY'] - df_new['TC_HOANTHANH']
        
        # Interaction: Nghỉ học nhiều + Rớt môn nhiều -> Rủi ro cực cao
        df_new['Risk_Score_Gap_Fail'] = df_new['Gap_Duration'] * df_new['TC_ROT']
        
        # Tỷ lệ rớt môn tích lũy (Cumulative Fail Ratio)
        df_new['Cum_TC_ROT'] = df_new.groupby('MA_SO_SV')['TC_ROT'].cumsum()
        df_new['Cum_TC_DANGKY'] = df_new.groupby('MA_SO_SV')['TC_DANGKY'].cumsum()
        df_new['Fail_Ratio_Accumulated'] = df_new['Cum_TC_ROT'] / (df_new['Cum_TC_DANGKY'] + 1e-5)

    # Dọn dẹp: Xóa các cột tạm trung gian nếu muốn
    cols_to_drop = ['Sem_Mean_GPA', 'Sem_Std_GPA'] 
    df_new = df_new.drop(columns=[c for c in cols_to_drop if c in df_new.columns])
    
    print("✅ Hoàn tất! Đã thêm {} biến mới.".format(len(df_new.columns) - len(df.columns)))
    return df_new


In [18]:
df = generate_advanced_features(df) 

🚀 Bắt đầu Feature Engineering nâng cao...
- Đang tính toán xu hướng học tập (Trend)...
- Đang tính toán so sánh với bạn đồng lứa (Z-Score)...
- Đang tạo các biến tương tác (Interaction)...
✅ Hoàn tất! Đã thêm 5 biến mới.


In [20]:
df.head()

,MA_SO_SV,HOC_KY,CPA,GPA,TC_DANGKY,TC_HOANTHANH,NAM_TUYENSINH,PTXT,TOHOP_XT,DIEM_TRUNGTUYEN,DIEM_CHUAN,GPA_Last_3_Avg,GPA_Volatility_Accumulated,GPA_Trend_Slope,GPA_ZScore,GPA_Percentile
0,00003e092652,HK1 2023-2024,1.64,1.97,18,15,2023,100,A00,21.32,20.25,1.970000,0.000000,0.00,-0.670752,0.235160
1,00003e092652,HK2 2023-2024,1.53,2.05,18,13,2023,100,A00,21.32,20.25,2.010000,0.056569,0.08,-0.528498,0.318245
2,000e15519006,HK1 2021-2022,3.85,3.85,9,9,2021,1,D07,23.84,22.43,3.850000,0.000000,0.00,3.031224,0.992203
3,000e15519006,HK1 2022-2023,2.83,2.98,21,21,2021,1,D07,23.84,22.43,3.415000,0.615183,-0.87,1.315777,0.906335
4,000e15519006,HK1 2023-2024,1.50,2.73,20,14,2021,1,D07,23.84,22.43,3.186667,0.587906,-0.56,0.886337,0.825662


In [ ]:
features = [
    'CPA', 'GPA', 'TC_DANGKY', 'NAM_TUYENSINH',
    'PTXT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN',
    'GPA_Last_3_Avg', 'GPA_Volatility_Accumulated',
    'GPA_Trend_Slope', 'GPA_ZScore', 'GPA_Percentile'
]
target = 'TC_HOANTHANH'
features

['CPA',
 'GPA',
 'TC_DANGKY',
 'NAM_TUYENSINH',
 'PTXT',
 'DIEM_TRUNGTUYEN',
 'DIEM_CHUAN',
 'GPA_Last_3_Avg',
 'GPA_Volatility_Accumulated',
 'GPA_Trend_Slope',
 'GPA_ZScore',
 'GPA_Percentile']

In [101]:
features = [
    'CPA', 'GPA', 'TC_DANGKY', 'NAM_TUYENSINH',
    'PTXT', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN',
    'GPA_Last_3_Avg', 'GPA_Volatility_Accumulated',
    'GPA_Trend_Slope', 'GPA_ZScore', 'GPA_Percentile'
]

In [104]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

features = ['GPA', 'CPA', 'TC_DANGKY', 'DIEM_TRUNGTUYEN', 'DIEM_CHUAN'] 
target = 'TC_HOANTHANH'

In [103]:
X_train, y_train = df[features], df[target]
X_valid, y_valid = df[features], df[target]

# ==========================================
# BƯỚC 1: TEST BASELINE (Dùng giá trị trung bình)
# ==========================================
y_train_mean = y_train.mean()
y_pred_baseline = np.full_like(y_valid, fill_value=y_train_mean)

# RMSE Baseline
rmse_baseline = np.sqrt(mean_squared_error(y_valid, y_pred_baseline))

# ==========================================
# BƯỚC 2: TRAIN & TEST LINEAR REGRESSION
# ==========================================
model = LinearRegression()
model.fit(X_train, y_train)

# Dự báo và làm tròn vì TC_HOANTHANH phải là số nguyên [cite: 44]
y_pred_model = model.predict(X_valid)
y_pred_model_int = np.round(y_pred_model).astype(int)

# RMSE Model
rmse_model = np.sqrt(mean_squared_error(y_valid, y_pred_model_int))

print(f"--- KẾT QUẢ SO SÁNH RMSE ---")
print(f"Baseline RMSE: {rmse_baseline:.4f}")
print(f"Linear Regression RMSE: {rmse_model:.4f}")

--- KẾT QUẢ SO SÁNH RMSE ---
Baseline RMSE: 6.6387
Linear Regression RMSE: 2.7920
